# Apple Stock

### Introduction:

We are going to use Apple's stock price.


### Step 1. Import the necessary libraries

In [0]:
from pyspark.sql.functions import *

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/09_Time_Series/Apple_Stock/appl_1980_2014.csv)

In [0]:
my_df=spark.read.option('inferschema',True).csv('/mnt/adlstestcontainer/csvfiles/appl_1980_2014.csv',header=True,sep=',')

### Step 3. Assign it to a variable apple

In [0]:
apple=my_df

### Step 4.  Check out the type of the columns

In [0]:
apple.printSchema()

### Step 5. Transform the Date column as a datetime type

In [0]:
apple = apple.withColumn('Date', to_timestamp(apple['Date'], "yyyy-MM-dd HH:mm:ss’"))

In [0]:
apple.printSchema()

### Step 6.  Set the date as the index

### Step 7.  Is there any duplicate dates?

In [0]:
apple.show(40)

In [0]:
apple.groupBy("date").count().where("count > 1").drop("count").show()
#No duplicates

### Step 8.  Ops...it seems the index is from the most recent date. Make the first entry the oldest date.

In [0]:
apple.orderBy(desc('date'))
apple.show()

### Step 9. Get the last business day of each month

In [0]:
# Group the data by year and month
grouped_data = apple.groupBy(year("date").alias("year"), month("date").alias("month"))
# Get the last day of each month
last_day_of_month = grouped_data.agg(last_day(max("date")).alias("last_day"))
# Get the last business day of each month
last_business_day_of_month = last_day_of_month.withColumn("day", dayofmonth(col("last_day")))
#excluding sundays
last_business_day_of_month = last_business_day_of_month.filter(expr("dayofweek(last_day) < 6"))
# Show the result
last_business_day_of_month.show(50)


### Step 10.  What is the difference in days between the first day and the oldest

In [0]:
apple.agg(expr("datediff(dayofmonth(max(date)), dayofmonth(min(date)))")).show()

### Step 11.  How many months in the data we have?

### Step 12. Plot the 'Adj Close' value. Set the size of the figure to 13.5 x 9 inches

### BONUS: Create your own question and answer it.